In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 18 15:40:39 2020

@author: sadrachpierre
"""
import pandas as pd 
import numpy as np 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
df = pd.read_csv("winemag-data-130k-v2.csv").sample(n=5000, random_state = 42)





In [2]:

del df['Unnamed: 0']
print(df.head())
print(df.info())

         country                                        description  \
77718  Australia  Possibly a little sweet, this is a soft, easyg...   
67681     France  A soft, almost off dry wine that is full in th...   
69877      Spain  Generic white-fruit aromas of peach and apple ...   
46544         US  This is the winery's best Nebula in years. Whi...   
186           US  This is a very rich Pinot whose primary virtue...   

                  designation  points  price         province  \
77718                     NaN      83    5.0  Australia Other   
67681                 Réserve      85   12.0     Rhône Valley   
69877  Estate Grown & Bottled      86    9.0   Northern Spain   
46544                  Nebula      87   29.0       California   
186            Wiley Vineyard      88   40.0       California   

                      region_1       region_2        taster_name  \
77718  South Eastern Australia            NaN     Joe Czerwinski   
67681           Côtes du Rhône            NaN 

In [3]:
df['country_cat'] = df['country'].astype('category')
df['country_cat'] = df['country_cat'].cat.codes

df['province_cat'] = df['province'].astype('category')
df['province_cat'] = df['province_cat'].cat.codes

df['winery_cat'] = df['winery'].astype('category')
df['winery_cat'] = df['winery_cat'].cat.codes

df['variety_cat'] = df['variety'].astype('category')
df['variety_cat'] = df['variety_cat'].cat.codes

In [10]:
df_filter = df[df['price'] > 0].copy()
df_filter = df_filter[df_filter['price'] <= df_filter['price'].mean() + df_filter['price'].std() ].copy()

In [11]:
from sklearn.linear_model import LinearRegression


In [12]:
print("Correlation: ", df['points'].corr(df['price']))

Correlation:  0.4626124788483178


In [13]:
kf = KFold(n_splits=10, random_state = 42,shuffle=True)

y_pred = []
y_true = []

In [29]:
for train_index, test_index in kf.split(df_filter):
    df_test = df_filter.iloc[test_index]
    df_train = df_filter.iloc[train_index]
    
    X_train = np.array(df_train['points']).reshape(-1, 1)     
    y_train = np.array(df_train['price']).reshape(-1, 1)
    X_test = np.array(df_test['points']).reshape(-1, 1)  
    y_test = np.array(df_test['price']).reshape(-1, 1)
    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred.append(model.predict(X_test)[0])
    y_true.append(y_test[0])  

In [31]:
from sklearn.ensemble import RandomForestRegressor

kf = KFold(n_splits=10, random_state = 42,shuffle=True)

y_pred_rf = []
y_true_rf = []

features = ['points', 'country_cat', 'province_cat', 'winery_cat', 'variety_cat']

for train_index, test_index in kf.split(df_filter):
    df_test = df_filter.iloc[test_index]
    df_train = df_filter.iloc[train_index]
    
    X_train = np.array(df_train[features])
    y_train = np.array(df_train['price'])
    X_test = np.array(df_test[features])
    y_test = np.array(df_test['price'])
    model = RandomForestRegressor(n_estimators = 1000, max_depth = 1000, random_state = 42)
    model.fit(X_train, y_train)

    y_pred_rf.append(model.predict(X_test)[0])
    y_true_rf.append(y_test[0])    

print("Mean Square Error (Random Forest): ", mean_squared_error(y_pred_rf, y_true_rf))
print("Mean Square Error (Linear Regression): ", mean_squared_error(y_true, y_pred))

Mean Square Error (Random Forest):  120.2616258642222
Mean Square Error (Linear Regression):  113.98967982781444
